In [1]:
from newsapi import NewsApiClient
import pandas as pd
from datetime import date, timedelta
from newspaper import Article, Config
import nltk
from nltk.corpus import stopwords

today = date.today()
yesterday = today - timedelta(days = 1)
day_before_yesterday = today - timedelta(days = 2)
# Init
newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(   
                                          category='entertainment',
                                          language='en',
                                          page_size = 90,
                                          page= 1)

articles = top_headlines.get('articles',[])

init_df = pd.DataFrame(articles, columns = ['source','title','publishedAt','author','url'])

init_df['source'] = init_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

init_df['publishedAt'] = pd.to_datetime(init_df['publishedAt'])



filtered_df = init_df[(init_df['publishedAt'].dt.date == day_before_yesterday) | (init_df['publishedAt'].dt.date == yesterday)]


df = filtered_df.copy()

def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config = config)

    try:
        page.download()
        page.parse()
        return page.text
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'


df['content'] = df['url'].apply(full_content)
df['content'] = df['content'].str.replace('\n', ' ')
df = df[df['content'] != 'couldnt retrieve']



# Download the stopwords dataset
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


def count_words_without_stopwords(text):
    if isinstance(text, (str, bytes)):
        words = nltk.word_tokenize(str(text))
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return len(filtered_words)
    else:
        0

df['word_count'] = df['content'].apply(count_words_without_stopwords)
df


Error retrieving content from https://www.kmbc.com/article/actor-cole-brings-plenty-found-dead-in-johnson-county-kansas/60413615: Article `download()` failed with 451 Client Error: Forbidden for url: https://www.kmbc.com/article/actor-cole-brings-plenty-found-dead-in-johnson-county-kansas/60413615 on URL https://www.kmbc.com/article/actor-cole-brings-plenty-found-dead-in-johnson-county-kansas/60413615
Error retrieving content from https://www.usatoday.com/story/entertainment/celebrities/2024/04/05/suki-waterhouse-robert-pattinson-child-angel/73216975007/: Article `download()` failed with 403 Client Error: Unknown Error for url: https://eu.usatoday.com/story/entertainment/celebrities/2024/04/05/suki-waterhouse-robert-pattinson-child-angel/73216975007/ on URL https://www.usatoday.com/story/entertainment/celebrities/2024/04/05/suki-waterhouse-robert-pattinson-child-angel/73216975007/
Error retrieving content from https://www.koimoi.com/box-office/maidaan-box-office-day-1-advance-booking-5

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/gitpod/nltk_data...


,source,title,publishedAt,author,url,content,word_count
0,YouTube,"WrestleMania XL Kickoff: April 5, 2024 - WWE",2024-04-05 22:10:57+00:00,None,https://www.youtube.com/watch?v=ewNnyLV5IEs,,0
2,The Atlantic,Why Beyoncé keeps reinventing herself - The At...,2024-04-05 21:52:00+00:00,The Atlantic,https://www.theatlantic.com/newsletters/archiv...,A conversation with Spencer Kornhaber about al...,979
3,Deadline,'Quantum Leap' Canceled By NBC After 2 Seasons...,2024-04-05 21:30:00+00:00,Nellie Andreeva,http://deadline.com/2024/04/quantum-leap-cance...,EXCLUSIVE: NBC has decided not to proceed with...,337
4,News18,Shweta Bachchan Gets Into A Debate With Jaya B...,2024-04-05 20:50:00+00:00,Kashvi Raj Singh,https://www.news18.com/movies/shweta-bachchan-...,The latest episode of Navya Naveli Nanda’s pod...,246
5,New York Post,Sean ‘Diddy’ Combs’ son accused of sexual assa...,2024-04-05 20:42:00+00:00,Megan Palin,https://nypost.com/2024/04/05/us-news/sean-did...,Sean “Diddy” Combs’ son has been accused in a ...,660
...,...,...,...,...,...,...,...
85,The Times of India,Ramayana: Photos of Arun Govil as Dashrath wit...,2024-04-05 01:11:00+00:00,TOI Entertainment Desk,https://timesofindia.indiatimes.com/entertainm...,It has been only a few days since director Nit...,271
86,Wrestlingheadlines.com,More Legends Spotted In Philadelphia Ahead of ...,2024-04-05 01:08:25+00:00,Joey G.,https://wrestlingheadlines.com/more-legends-sp...,More big names have been spotted in Philadelph...,83
87,Fox Business,Rebel Wilson claims agency 'liked me fat' afte...,2024-04-05 00:42:00+00:00,Janelle Ash,https://www.foxbusiness.com/entertainment/rebe...,Rebel Wilson reportedly secured a $10 million ...,396
88,Deadline,"‘Game Of Thrones’ Joseph Gatt Hits LA, DA, & P...",2024-04-04 23:25:00+00:00,Dominic Patten,http://deadline.com/2024/04/joseph-gatt-sues-l...,Almost two years after Joseph Gatt was arreste...,685


In [2]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# download vader lexicon

nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

def get_sentiment(row):
    sentiment_scores = sid.polarity_scores(row)
    compound_score = sentiment_scores['compound']

    if compound_score >= 0.05:
        sentiment = 'Positive'
    elif compound_score <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    return sentiment, compound_score

df[['Sentiment', 'Compound_Score']] = df['content'].astype(str).apply(lambda x: pd.Series(get_sentiment(x)))

df

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gitpod/nltk_data...


,source,title,publishedAt,author,url,content,word_count,Sentiment,Compound_Score
0,YouTube,"WrestleMania XL Kickoff: April 5, 2024 - WWE",2024-04-05 22:10:57+00:00,None,https://www.youtube.com/watch?v=ewNnyLV5IEs,,0,Neutral,0.0000
2,The Atlantic,Why Beyoncé keeps reinventing herself - The At...,2024-04-05 21:52:00+00:00,The Atlantic,https://www.theatlantic.com/newsletters/archiv...,A conversation with Spencer Kornhaber about al...,979,Positive,0.9975
3,Deadline,'Quantum Leap' Canceled By NBC After 2 Seasons...,2024-04-05 21:30:00+00:00,Nellie Andreeva,http://deadline.com/2024/04/quantum-leap-cance...,EXCLUSIVE: NBC has decided not to proceed with...,337,Positive,0.9855
4,News18,Shweta Bachchan Gets Into A Debate With Jaya B...,2024-04-05 20:50:00+00:00,Kashvi Raj Singh,https://www.news18.com/movies/shweta-bachchan-...,The latest episode of Navya Naveli Nanda’s pod...,246,Negative,-0.3208
5,New York Post,Sean ‘Diddy’ Combs’ son accused of sexual assa...,2024-04-05 20:42:00+00:00,Megan Palin,https://nypost.com/2024/04/05/us-news/sean-did...,Sean “Diddy” Combs’ son has been accused in a ...,660,Negative,-0.9968
...,...,...,...,...,...,...,...,...,...
85,The Times of India,Ramayana: Photos of Arun Govil as Dashrath wit...,2024-04-05 01:11:00+00:00,TOI Entertainment Desk,https://timesofindia.indiatimes.com/entertainm...,It has been only a few days since director Nit...,271,Positive,0.9906
86,Wrestlingheadlines.com,More Legends Spotted In Philadelphia Ahead of ...,2024-04-05 01:08:25+00:00,Joey G.,https://wrestlingheadlines.com/more-legends-sp...,More big names have been spotted in Philadelph...,83,Positive,0.8519
87,Fox Business,Rebel Wilson claims agency 'liked me fat' afte...,2024-04-05 00:42:00+00:00,Janelle Ash,https://www.foxbusiness.com/entertainment/rebe...,Rebel Wilson reportedly secured a $10 million ...,396,Positive,0.9767
88,Deadline,"‘Game Of Thrones’ Joseph Gatt Hits LA, DA, & P...",2024-04-04 23:25:00+00:00,Dominic Patten,http://deadline.com/2024/04/joseph-gatt-sues-l...,Almost two years after Joseph Gatt was arreste...,685,Negative,-0.9891
